In [2]:
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from pylab import *
from keras.callbacks import LearningRateScheduler
from keras import models
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
from keras.applications.vgg19 import VGG19
import itertools
from sklearn.metrics import confusion_matrix
from keras import optimizers
from PIL import Image
import pandas as pd
import os


In [3]:
def step_decay_schedule(initial_lr=1e-3, decay_factor=0.75, step_size=10):
        def schedule(epoch):
            return initial_lr * (decay_factor ** np.floor(epoch / step_size))
        return LearningRateScheduler(schedule)

In [4]:
 vgg_conv = VGG19(weights='imagenet', include_top=False, input_shape=(100, 100, 3))

80142336/80134624 [==============================] - 212s 3us/step


In [11]:
    model = models.Sequential()
    
    for layer in vgg_conv.layers[:-5]:
        layer.trainable = False

    # Check the trainabl status of the individual layers
    for layer in vgg_conv.layers:
        print(layer, layer.trainable)


    model.add(vgg_conv)
    model.summary()
    model.add(layers.Flatten())
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dropout(0.50))
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dropout(0.50))
    model.add(layers.Dense(2, activation='softmax'))

    optimizer = optimizers.Adagrad(lr=0.001, epsilon=None, decay=0.0)
    model.compile(optimizer=optimizer,
                  loss="mean_squared_error",
                  metrics=["accuracy"])

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x0000014DC2C5A688> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000014DC2B118C8> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000014DC305CE08> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x0000014DC34CB648> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000014DC34C5A48> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000014DC34DB208> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x0000014DC34D5B88> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000014DC34DFBC8> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000014DC34EB648> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000014DC34F6DC8> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000014DC34F2C48> False
<tensorflow.python.

In [12]:
lr_sched = step_decay_schedule(initial_lr=1e-4, decay_factor=0.75, step_size=2)

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
TRAINING_DIR = 'Image_Dataset/Train'
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,target_size=(100,100),batch_size=5,class_mode='categorical')
VALIDATION_DIR = 'Image_Dataset/Test'
validation_datagen = ImageDataGenerator(rescale=1.0/255.0);
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,target_size=(100,100),batch_size=10,class_mode='categorical');

Found 1599 images belonging to 2 classes.
Found 401 images belonging to 2 classes.


In [14]:
history = model.fit(
      train_generator,
      steps_per_epoch=100,  # 2000 images = batch_size * steps
      epochs=10,
      validation_data=validation_generator,
      validation_steps=10,
      callbacks=[lr_sched])

Epoch 1/10
100/100 [==============================] - 161s 2s/step - loss: 0.2969 - accuracy: 0.6320 - val_loss: 0.2069 - val_accuracy: 0.7300 - lr: 1.0000e-04
Epoch 2/10
100/100 [==============================] - 159s 2s/step - loss: 0.2668 - accuracy: 0.6340 - val_loss: 0.2174 - val_accuracy: 0.6900 - lr: 1.0000e-04
Epoch 3/10
100/100 [==============================] - 156s 2s/step - loss: 0.2820 - accuracy: 0.6032 - val_loss: 0.2489 - val_accuracy: 0.6400 - lr: 7.5000e-05
Epoch 4/10
100/100 [==============================] - 157s 2s/step - loss: 0.2599 - accuracy: 0.6120 - val_loss: 0.2407 - val_accuracy: 0.6200 - lr: 7.5000e-05
Epoch 5/10
100/100 [==============================] - 155s 2s/step - loss: 0.2790 - accuracy: 0.6160 - val_loss: 0.2486 - val_accuracy: 0.6000 - lr: 5.6250e-05
Epoch 6/10
100/100 [==============================] - 156s 2s/step - loss: 0.2741 - accuracy: 0.5980 - val_loss: 0.2215 - val_accuracy: 0.7000 - lr: 5.6250e-05
Epoch 7/10
100/100 [====================